# What Dis

2010–2016 crime data for cities throughout the United States as curated by the FBI

[source](https://ucr.fbi.gov/crime-in-the-u.s/)

# Imports and Definitions

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def to_val(cell):
    val = cell.get_text().strip().replace(',', '')
    return float(val) if val != '' else np.nan

In [4]:
typos = { # good job, FBI
    "massachuetts": "massachusetts"
}

In [5]:
years = [str(year) for year in range(2009, 2016+1)]

# Scraping

In [6]:
data = []
with requests.Session() as session:
    for year in years:
        print("scraping {year} data for".format(year=year), end=' ')
        if year == '2009':
            states_list_url = "https://www2.fbi.gov/ucr/cius2009/data/table_08.html"
            state_anchors = BeautifulSoup(session.get(states_list_url).content, 'html.parser') \
                                  .find_all(id='page_content')[-1]                             \
                                  .find(class_='secondarywrapper')                             \
                                  .find_all('a')
        else:
            base_url = "https://ucr.fbi.gov/crime-in-the-u.s/{year}/crime-in-the-u.s.-{year}".format(year=year)
            states_list_url = BeautifulSoup(session.get(base_url).content, 'html.parser') \
                                  .find(id='quick_finds')                                 \
                                  .find_next('div')                                       \
                                  .find_next('a')                                         \
                                  .get('href')
            # "secardary" is not a typo on my end
            state_anchors = BeautifulSoup(session.get(states_list_url).content, 'html.parser') \
                                  .find_all(id='page_content')[-1]                             \
                                  .find(class_='secardarywrapper')                             \
                                  .find_all('a')
        for state_anchor in state_anchors:
            state = state_anchor.get_text().strip().lower()
            if state in typos:
                state = typos[state]
            print(state, end=', ')
            state_url = state_anchor.get('href')
            if year == '2009': # 2009 uses relative hrefs
                state_url = states_list_url[:-13] + state_url
                
            table_body = BeautifulSoup(session.get(state_url).content, 'html.parser') \
                              .find_all('table', class_='data')[-1]                   \
                              .find('tbody')
            if year == '2009': # the formatting for 2009 is atrocious
                year_data = []
                for row in table_body.find_all('tr'):
                    cells = row.find_all('td')
                    year_data.append(pd.Series([
                        state,
                        row.find('th').get_text().strip().rstrip("0123456789").lower(), # city
                        year,
                        to_val(cells[0]), # population
                        to_val(cells[2]), # murder
                        to_val(cells[3]), # forcible rape
                        to_val(cells[4]), # robbery
                        to_val(cells[5]), # aggravated assault
                        to_val(cells[7]), # burglary
                        to_val(cells[8]), # larceny or theft
                        to_val(cells[9]), # motor vehicle theft
                        to_val(cells[10]) # arson
                    ]).to_frame().transpose())
                year_df = pd.concat(year_data, ignore_index=True)
                year_df.columns = ['state',
                    'city',
                    'year',
                    'population',
                    'murder',
                    'forcible rape',
                    'robbery',
                    'aggravated assault',
                    'burglary',
                    'larceny or theft',
                    'motor vehicle theft',
                    'arson'
                ]
                data.append(year_df)
            elif year in ['2010', '2011', '2012']:
                data.append(pd.DataFrame({
                    'state': state,
                    'city': [cell.get_text().strip().rstrip("0123456789").lower()
                             for cell in table_body.find_all(class_='group0')],
                    'year': year,
                    'population': [to_val(cell)
                             for cell in table_body.find_all(class_='group1')],
                    'murder': [to_val(cell)
                             for cell in table_body.find_all(class_='group3')],
                    'forcible rape': [to_val(cell)
                             for cell in table_body.find_all(class_='group4')],
                    'robbery': [to_val(cell)
                             for cell in table_body.find_all(class_='group5')],
                    'aggravated assault': [to_val(cell)
                             for cell in table_body.find_all(class_='group6')],
                    'burglary': [to_val(cell)
                             for cell in table_body.find_all(class_='group8')],
                    'larceny or theft': [to_val(cell)
                             for cell in table_body.find_all(class_='group9')],
                    'motor vehicle theft': [to_val(cell)
                             for cell in table_body.find_all(class_='group10')],
                    'arson': [to_val(cell)
                             for cell in table_body.find_all(class_='group11')]
                }))
            else: # the FBI revised their definition of rape in 2013, and added an extra column to the table
                year_data = pd.DataFrame({
                    'state': state,
                    'city': [cell.get_text().strip().rstrip("0123456789").lower()
                             for cell in table_body.find_all(class_='group0')],
                    'year': year,
                    'population': [to_val(cell)
                             for cell in table_body.find_all(class_='group1')],
                    'murder': [to_val(cell)
                             for cell in table_body.find_all(class_='group3')],
                    'rape (revised)': [to_val(cell)
                             for cell in table_body.find_all(class_='group4')],
                    'rape (legacy)': [to_val(cell)
                             for cell in table_body.find_all(class_='group5')],
                    'robbery': [to_val(cell)
                             for cell in table_body.find_all(class_='group6')],
                    'aggravated assault': [to_val(cell)
                             for cell in table_body.find_all(class_='group7')],
                    'burglary': [to_val(cell)
                             for cell in table_body.find_all(class_='group9')],
                    'larceny or theft': [to_val(cell)
                             for cell in table_body.find_all(class_='group10')],
                    'motor vehicle theft': [to_val(cell)
                             for cell in table_body.find_all(class_='group11')],
                    'arson': [to_val(cell)
                             for cell in table_body.find_all(class_='group12')]
                })
                year_data['forcible rape'] = pd.Series([legacy if pd.isnull(revised) else revised
                                                        for revised, legacy
                                                        in zip(
                                                            year_data['rape (revised)'].values,
                                                            year_data['rape (legacy)'].values)],
                                                       index=year_data.index)
                year_data.drop(['rape (revised)', 'rape (legacy)'], axis=1, inplace=True)
                data.append(year_data)
        print('\n')

df = pd.concat(data, ignore_index=True).set_index(['state', 'city', 'year']).sort_index()

scraping 2009 data for alabama, alaska, arizona, arkansas, california, colorado, connecticut, delaware, district of columbia, florida, georgia, hawaii, idaho, illinois, indiana, iowa, kansas, kentucky, louisiana, maine, maryland, massachusetts, michigan, minnesota, mississippi, missouri, montana, nebraska, nevada, new hampshire, new jersey, new mexico, new york, north carolina, north dakota, ohio, oklahoma, oregon, pennsylvania, rhode island, south carolina, south dakota, tennessee, texas, utah, vermont, virginia, washington, west virginia, wisconsin, wyoming, 

scraping 2010 data for alabama, alaska, arizona, arkansas, california, colorado, connecticut, delaware, district of columbia, florida, georgia, hawaii, idaho, illinois, indiana, iowa, kansas, kentucky, louisiana, maine, maryland, massachusetts, michigan, minnesota, mississippi, missouri, montana, nebraska, nevada, new hampshire, new jersey, new mexico, new york, north carolina, north dakota, ohio, oklahoma, oregon, pennsylvania

In [7]:
df.to_pickle("fbi_crime_data.pkl")

In [8]:
df

aggravated assault arson burglary forcible rape  \
state   city        year                                                   
alabama abbeville   2009                  8   NaN       18             1   
                    2010                 20   NaN       19             0   
                    2011                 10     1       12             1   
                    2012                 18   NaN       16             1   
                    2013                  7   NaN       21             1   
                    2014                  7   NaN       22             1   
                    2015                  6   NaN       24             3   
                    2016                 10   NaN       12             1   
        adamsville  2009                  7   NaN       33             1   
                    2010                  5   NaN       21             3   
                    2011                 26     2       61             2   
                    2012                 20   NaN       49             1   
                    2013                 11   NaN       58             0   
                    2014                 19   NaN       43             5   
                    2015                 16   NaN       38             3   
                    2016                  9   NaN       33             0   
        addison     2009                  4   NaN        4             1   
                    2010                  0   NaN        3             0   
                    2011                  1     1        9             0   
                    2012                  7   NaN        5             0   
                    2013                  0   NaN        6             1   
                    2014                  1   NaN        6             0   
                    2015                  4   NaN       11             0   
                    2016                  1   NaN        1             0   
        alabaster   2009                 27   NaN       82             2   
                    2010                 26   NaN       22             0   
                    2011                 46     1       86             1   
                    2012                 41   NaN       65             2   
                    2013                 31   NaN       70             2   
                    2014                 37   NaN       68             9   
...                                     ...   ...      ...           ...   
wyoming thermopolis 2011                  0     0        3             0   
                    2012                  6     1        6             0   
                    2013                  2     0       14             0   
                    2014                  0     3        5             1   
                    2015                  0     0        1             1   
                    2016                  3     0        3             0   
        torrington  2009                  9     0       29             2   
                    2010                 17     0       28             1   
                    2011                  7     0       20             2   
                    2012                 26     0       33             0   
                    2013                 17     0       33             1   
                    2014                 20     0       29             1   
                    2015                 18     0       24             0   
                    2016                 25     0       19             0   
        wheatland   2009                  4     0       26             0   
                    2010                  2     0       51             2   
                    2011                  4     0       12             1   
                    2012                  3     0       15             1   
                    2013                 11     0       21             0   
                    2014                  5     0       32             2 